# Parallel Interaction on JudgeBench with Position Swapping

## Imports

In [3]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm
from collections import Counter

## Data

In [4]:
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")

df = pd.DataFrame(JudgeBench_Claude)

df_sampled = df.iloc[:270].sample(n=100, random_state=42).reset_index(drop=True)

df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [5]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

## System Design

In [6]:
agent_1_system_message =f"""
You are a Scientific-Agent with a background in academic research and critical thinking.
Your task is to assess which of Response A or Response B is more factually correct. 
Focus on whether the statements are logically sound, consistent with known facts and science.
Explain your decision using about 200 words.
Always begin your output with "As a Scientific-Agent, I think ..."
Always end your output with stating if Response A or B is more factual correct by using a JSON-object with the follwing format: {{"response": A/B}}
"""

agent_2_system_message =f"""
You are a Logical-Agent specializing in evaluating argument consistency, clarity and coherence.
Your task is to analyze Response A and Response B in terms of internal logical structure and factual plausibility.
Look for contradictions, fallacies, and misleading reasoning, even if subtle.
Explain your decision using about 200 words.
Always begin your output with: "As a Logical-Agent I think ..."
Always end your output with stating if Response A or B is more factual correct by using a JSON-object with the follwing format: {{"response": A/B}}
"""

agent_3_system_message ="""
You are a Domain-Expert-Agent with applied experience in the relevant field.
Your task is to judge which of Response A or Response B reflects real-world knowledge and practical correctness more accurately.
Focus on practical validity, typical use cases, and whether the information would hold in applied scenarios.
Explain your decision using about 200 words.
Always begin your output with: "As a Domain-Expert-Agent I think ..."
Always end your output with stating if Response A or B is more factual correct by using a JSON-object with the follwing format: {{"response": A/B}}
"""

In [7]:
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    )

agent_1 = ConversableAgent(
    "Scientific-Agent",
    llm_config={"config_list": config_list},
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)
agent_2 = ConversableAgent(
    "Logical-Agent",
    llm_config={"config_list": config_list},
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)
agent_3 = ConversableAgent(
    "Domain-Expert",
    llm_config={"config_list": config_list},
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

In [8]:
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    agents = [agent_1, agent_2, agent_3]
    decisions = []

    pattern = r'"response"\s*:\s*"?([AB])"?'

    ground_truth = "A" if "A>" in label else "B"

    for agent in agents:
            result = initializer.initiate_chat(agent, message=message, max_turns=1)
            result_str = str(result)
            print(result_str)

            match = re.search(pattern, result_str)
            decision = match.group(1) if match else "X"
            print(decision)
            decisions.append(decision)

    if "X" in decisions:
        return {
            "system_decision": "X",
            "ground_truth": ground_truth,
            "is_correct": False
        }

    system_decision = Counter(decisions).most_common(1)[0][0]
    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }


In [9]:
num_rows = 10

df_final_subset = df_final.head(num_rows)

# normal position
results_1 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/parallel_1.csv', index=False)

accuracy_1 = results_1_df["is_correct"].mean()
print(f"Accuracy_1: {accuracy_1:.2%}")

position_distribution_1 = results_1_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_1: {position_distribution_1}")

# swapped position
results_2 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"], # positions swap 
        response_B=row["response_A"], # positions swap 
        label=row["label"],
    )

    # position swap back
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/parallel_2.csv', index=False)

accuracy_2 = results_2_df["is_correct"].mean()
print(f"Accuracy_2: {accuracy_2:.2%}")

position_distribution_2 = results_2_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_2: {position_distribution_2}")

Progress:   0%|          | 0/10 [00:00<?, ?it/s]

initializer (to Scientific-Agent):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mol

Progress:  10%|█         | 1/10 [00:07<01:10,  7.83s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.\n(A) .1002 : .3002 : .4002\n(B) .3582 : .4582 : .0582\n(C) .2582 : .4998 : .3092\n(D) .1582 : .6008 : .2092\n(E) .1582 : .4998 : .1092\n(F) .2002 : .4002 : .1092\n(G) .1582 : .3998 : .2592\n(H) .2502 : .4502 : .1502\n(I) .2082 : .5998 : .1592\n(J) .1802 : .4802 : .2802\nIf you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.\nLet's think step by step.\n\n    Response A: Let's approach this step-by-step:\n\n1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.\n\n2) We need to find the relative rates, so we'll use the formula: \n 

Progress:  20%|██        | 2/10 [00:14<00:55,  6.95s/it]

...

ChatResult(chat_id=None, chat_history=[{'content': " \n    Question: A radioactive sample contains two different isotopes, A and B. A has a half-life of 3 days, and B has a half-life of 6 days. Initially in the sample there are twice as many atoms of A as of B. In how many days will the ratio of the number of atoms of A to B be reversed?\n(A) 15.0\n(B) 18.0\n(C) 12.0\n(D) 24.0\n(E) 9.0\n(F) 6.0\n(G) 21.0\n(H) 27.0\n(I) 10.5\n(J) 3.0\nIf you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.\nLet's think step by step.\n\n    Response A: Let's approach this step-by-step:\n\n1) Let's say initially there are 2x atoms of A and x atoms of B.\n\n2) After t days, the number of atoms of A will be: 2x * (1/2)^(t/3)\n   And the number of atoms of B will be: x * (1/2)^(t/6)\n\n3) We want to find when the ratio of A to B becomes 1/2. So we

Progress:  90%|█████████ | 9/10 [01:08<00:08,  8.06s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Question: Compute the sample standard deviation of ${9, 14, 5, 4, -20, -13, -5, 13}$. Please put your final answer in a $\\\\boxed{}$.\n\n    Response A: Let's approach this step-by-step:\n\n1) First, we need to calculate the mean (average) of the dataset.\n   Mean = (9 + 14 + 5 + 4 + (-20) + (-13) + (-5) + 13) ÷ 8 = 7 ÷ 8 = 0.875\n\n2) Now, we subtract the mean from each data point and square the result:\n   (9 - 0.875)² = 66.015625\n   (14 - 0.875)² = 171.390625\n   (5 - 0.875)² = 16.890625\n   (4 - 0.875)² = 9.765625\n   (-20 - 0.875)² = 435.890625\n   (-13 - 0.875)² = 192.390625\n   (-5 - 0.875)² = 34.515625\n   (13 - 0.875)² = 147.015625\n\n3) We sum these squared differences:\n   66.015625 + 171.390625 + 16.890625 + 9.765625 + 435.890625 + 192.390625 + 34.515625 + 147.015625 = 1073.875\n\n4) We divide this sum by (n-1), where n is the number of data points. Here, n = 8.\n   1073.875 ÷ 7 = 153.410714\n\n5) Finally, we take

Progress: 100%|██████████| 10/10 [01:17<00:00,  7.76s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Question: You are given a 0-indexed integer array nums.\nA subsequence of nums having length k and consisting of indices i_0 < i_1 < ... < i_k-1 is balanced if the following holds:\n\nnums[i_j] - nums[i_j-1] >= i_j - i_j-1, for every j in the range [1, k - 1].\n\nA subsequence of nums having length 1 is considered balanced.\nReturn an integer denoting the maximum possible sum of elements in a balanced subsequence of nums.\nA subsequence of an array is a new non-empty array that is formed from the original array by deleting some (possibly none) of the elements without disturbing the relative positions of the remaining elements.\n \nExample 1:\n\nInput: nums = [3,3,5,6]\nOutput: 14\nExplanation: In this example, the subsequence [3,5,6] consisting of indices 0, 2, and 3 can be selected.\nnums[2] - nums[0] >= 2 - 0.\nnums[3] - nums[2] >= 3 - 2.\nHence, it is a balanced subsequence, and its sum is the maximum among the balanced subs